# Decision Trees
## Introduction
"A decision tree is a flowchart-like structure in which each internal node represents a test of an attribute, each branch represents an outcome of that test and each leaf node represents class label (a decision taken after testing all attributes in the path from the root to the leaf). Each path from the root to a leaf can also be represented as a classification rule."  
[Wikipedia: Decision Tree](https://en.wikipedia.org/wiki/Decision_tree)  

## Weather Example
The weather example from Witten / Eibe determines whether to play tennis on a given day, or not. We implement a basic version of the ID3 algorithm to create a decision tree that allows to predict new instances based on the earlier measurements.

## Algorithm
The ID3 algorithm uses a divide and conquer top-down approach that works as follows:

1. Choose the best attribute as the node. Create a branch for each possible attribute value
2. Part the instances into subsets. One for each branch from the root node
3. Repeat steps 1 and 2 recursively for each branch, only the instances belonging to the respective branch be considered. 
Stop if all instances belong to the same class


<img src="Decision_Tree_Weather.gif">

# Imports

In [1]:
from collections import defaultdict, Counter
import math
from pprint import pprint

# Data

In [2]:
attribute_names = ("Play", "Outlook", "Temp", "Humidity", "Windy")

data = [("No", "Sunny", "Hot", "High", "False"),
        ("No", "Sunny", "Hot", "High", "True"),
        ("Yes", "Overcast", "Hot", "High", "False"),
        ("Yes", "Rain", "Mild", "High", "False"),
        ("Yes", "Rain", "Cool", "Normal", "False"),
        ("No", "Rain", "Cool", "Normal", "True"),
        ("Yes", "Overcast", "Cool", "Normal", "True"),
        ("No", "Sunny", "Mild", "High", "False"),
        ("Yes", "Sunny", "Cool", "Normal", "False"),
        ("Yes", "Rain", "Mild", "Normal", "False"),
        ("Yes", "Sunny", "Mild", "Normal", "True"),
        ("Yes", "Overcast", "Mild", "High", "True"),
        ("Yes", "Overcast", "Hot", "Normal", "False"),
        ("No", "Rain", "Mild", "High", "True")]

new_data = ("?", "Rain", "Mild", "Normal", "False")

pprint(attribute_names)
pprint(data)

('Play', 'Outlook', 'Temp', 'Humidity', 'Windy')
[('No', 'Sunny', 'Hot', 'High', 'False'),
 ('No', 'Sunny', 'Hot', 'High', 'True'),
 ('Yes', 'Overcast', 'Hot', 'High', 'False'),
 ('Yes', 'Rain', 'Mild', 'High', 'False'),
 ('Yes', 'Rain', 'Cool', 'Normal', 'False'),
 ('No', 'Rain', 'Cool', 'Normal', 'True'),
 ('Yes', 'Overcast', 'Cool', 'Normal', 'True'),
 ('No', 'Sunny', 'Mild', 'High', 'False'),
 ('Yes', 'Sunny', 'Cool', 'Normal', 'False'),
 ('Yes', 'Rain', 'Mild', 'Normal', 'False'),
 ('Yes', 'Sunny', 'Mild', 'Normal', 'True'),
 ('Yes', 'Overcast', 'Mild', 'High', 'True'),
 ('Yes', 'Overcast', 'Hot', 'Normal', 'False'),
 ('No', 'Rain', 'Mild', 'High', 'True')]


# Decision Tree
## Choosing the best attribute to split

In [3]:
def entropy(instances, class_index=0, value_name=None):
    '''Calculate the entropy of attribute in position class_index for the list of instances.'''
    num_instances = len(instances)

    ## there is no uncertainty if there are at most 1 instance
    if num_instances <= 1:
        return 0

    ## instantiates a dict with key: 0
    value_counts = defaultdict(int)

    for instance in instances:
        # defaultdict creates a new key/value pair for every value in
        # instance and increments it starting from zero
        value_counts[instance[class_index]] += 1
    
    num_values = len(value_counts)

    ## if the value of every instance is the same there is no uncertainty
    if num_values <= 1:
        return 0
    
    attribute_entropy = 0.0
    
    ## Calculate the attribute_entropy based on all the entries in value_counts.
    ## Iterate over value_counts, calculate the probability and then determine the entropy for that value.
    ## Combine child_entropy for all values to determine the overall entropy
    
    ## value_counts looks like this after the first run:
    ## {'No': 5, 'Yes': 9}
    
    ## you can calculate entropy for 1 child attribute as follows 
    ## child_entropy = value_probability * math.log(value_probability, 2)
    
    for value in value_counts:
        value_probability = value_counts[value] / num_instances
        child_entropy = value_probability * math.log(value_probability, 2)
        attribute_entropy -= child_entropy
    ##    
    return attribute_entropy

In [4]:
## Test the entropy function

print('Expected Entropy: 0.9402859586706309')
print('Your Entropy:    ', entropy(data))

Expected Entropy: 0.9402859586706309
Your Entropy:     0.9402859586706309


In [5]:
def information_gain(instances, parent_index, class_index=0):
    '''Return the information gain of splitting the instances based on the attribute parent_index'''
    
    
    ## determine the parent_entropy based on the input instances
    parent_entropy = entropy(instances, class_index)
    ##
    
    child_instances = defaultdict(list)
    '''
    append the instances for the given parent_index to the dict child_instances
    the structure should look like this:
    {key: [list with (instance tuples)]}
    
    {'Overcast': [('Yes', 'Overcast', 'Hot', 'High', 'False'), 
                ('Yes', 'Overcast', 'Cool', 'Normal', 'True'), 
                ('Yes', 'Overcast', 'Mild', 'High', 'True'), 
                ('Yes', 'Overcast', 'Hot', 'Normal', 'False')], 
    'Sunny': [('No', 'Sunny', 'Hot', 'High', 'False'), 
            ('No', 'Sunny', 'Hot', 'High', 'True'), 
            ('No', 'Sunny', 'Mild', 'High', 'False'), 
            ('Yes', 'Sunny', 'Cool', 'Normal', 'False'), 
            ('Yes', 'Sunny', 'Mild', 'Normal', 'True')], 
    'Rain': [('Yes', 'Rain', 'Mild', 'High', 'False'), 
            ('Yes', 'Rain', 'Cool', 'Normal', 'False'), 
            ('No', 'Rain', 'Cool', 'Normal', 'True'), 
            ('Yes', 'Rain', 'Mild', 'Normal', 'False'), 
            ('No', 'Rain', 'Mild', 'High', 'True')]}
    
    '''
    for instance in instances:
        child_instances[instance[parent_index]].append(instance)

    
    children_entropy = 0.0
    num_instances = len(instances)
    
    for child_value in child_instances:
        child_probability = len(child_instances[child_value]) / num_instances
        children_entropy += child_probability * entropy(child_instances[child_value], class_index, child_value)
    
    return parent_entropy - children_entropy

In [6]:
## Test information_gain

## list comprehension instead of for-loop
## the following for-loop would do the same
# information_gain_indexes = []
# for i in range(1, len(attribute_names)):
#    information_gain_indexes.append((information_gain(data, i), i))
    
information_gain_indexes = [(information_gain(data, i), i) for i in range(1, len(attribute_names))]


## sort descending with reverse=True
## if not specified otherwise the function sorts based on the first element of a tupel
sorted_information_gain_indexes = sorted(information_gain_indexes, reverse=True)


print('Expected Result')
print('Gain', 'Index', 'Attribute')
print(0.247, 1, 'Outlook')
print(0.152, 3, 'Humidity')
print(0.048, 4, 'Windy')
print(0.029, 2, 'Temp')
print()

print('Your result')
print("Gain", "Index", "Attribute")
for gain, i in sorted_information_gain_indexes:
    print('{:5.3f}  {:2} {}'.format(gain, i, attribute_names[i]))

Expected Result
Gain Index Attribute
0.247 1 Outlook
0.152 3 Humidity
0.048 4 Windy
0.029 2 Temp

Your result
Gain Index Attribute
0.247   1 Outlook
0.152   3 Humidity
0.048   4 Windy
0.029   2 Temp


In [7]:
def choose_best_attribute_index(instances, candidate_attribute_indexes, class_index=0):
    '''Return the index of the attribute that will provide the greatest information gain
    if instances were partitioned based on that attribute'''
    
    ## calculate the information_gain for all the attributes in candidate_attribute_indexes
    ## return the index of the attribute with the highest gain
    gains_and_indexes = sorted([(information_gain(instances, i), i) for i in candidate_attribute_indexes],
                               reverse=True)
    return gains_and_indexes[0][1]
    ##

In [8]:
## Test choose_best_attribute_index

candidate_attribute_indexes = [i for i in range(len(data[0])) if i != 0]

best_attribute_index = choose_best_attribute_index(data, candidate_attribute_indexes)
print('Expected result: 1')
print('Your result:    ', best_attribute_index)

Expected result: 1
Your result:     1


## splitting instances

In [9]:
def split_instances(instances, attribute_index):
    '''Returns a list of dictionaries, splitting a list of instances
        according to their values of a specified attribute index

    The key of each dictionary is a distinct value of attribute_index,
    and the value of each dictionary is a list representing
       the subset of instances that have that value for the attribute
    '''
    partitions = defaultdict(list)
    for instance in instances:
        partitions[instance[attribute_index]].append(instance)
    return partitions

In [10]:
partitions = split_instances(data, 1)
pprint(partitions)

defaultdict(<class 'list'>,
            {'Overcast': [('Yes', 'Overcast', 'Hot', 'High', 'False'),
                          ('Yes', 'Overcast', 'Cool', 'Normal', 'True'),
                          ('Yes', 'Overcast', 'Mild', 'High', 'True'),
                          ('Yes', 'Overcast', 'Hot', 'Normal', 'False')],
             'Rain': [('Yes', 'Rain', 'Mild', 'High', 'False'),
                      ('Yes', 'Rain', 'Cool', 'Normal', 'False'),
                      ('No', 'Rain', 'Cool', 'Normal', 'True'),
                      ('Yes', 'Rain', 'Mild', 'Normal', 'False'),
                      ('No', 'Rain', 'Mild', 'High', 'True')],
             'Sunny': [('No', 'Sunny', 'Hot', 'High', 'False'),
                       ('No', 'Sunny', 'Hot', 'High', 'True'),
                       ('No', 'Sunny', 'Mild', 'High', 'False'),
                       ('Yes', 'Sunny', 'Cool', 'Normal', 'False'),
                       ('Yes', 'Sunny', 'Mild', 'Normal', 'True')]})


## find the majority value

In [11]:
def majority_value(instances, class_index=0):
    '''Return the most frequent value of class_index in instances
       It may be easier to use the Counter that is already imported
       https://docs.python.org/3.1/library/collections.html#collections.Counter
    '''
    class_counts = Counter([instance[class_index] for instance in instances])
    return class_counts.most_common(1)[0][0]

In [12]:
## Test majority_value
print('Expected result: Yes')
print('Your result:    ', majority_value(data))


Expected result: Yes
Your result:     Yes


## building the decision tree

In [13]:
def create_decision_tree(instances,
                         candidate_attribute_indexes=None,
                         class_index=0,
                         default_class=None):
    '''Returns a new decision tree trained on a list of instances.

    The tree is constructed by recursively selecting and splitting instances based on
    the highest information_gain of the candidate_attribute_indexes.

    The class label is found in position class_index.

    The default_class is the majority value for the current node's parent in the tree.

    Derived from the simplified ID3 algorithm presented in Building Decision Trees in Python
        by Christopher Roach,
    http://www.onlamp.com/pub/a/python/2006/02/09/ai_decision_trees.html?page=3
    '''

    # if no candidate_attribute_indexes are provided,
    # assume that we will use all but the target_attribute_index
    # Note that None != [],
    # as an empty candidate_attribute_indexes list is a recursion stopping condition
    if candidate_attribute_indexes is None:
        candidate_attribute_indexes = [i
                                       for i in range(len(instances[0]))
                                       if i != class_index]
        # Note: do not use candidate_attribute_indexes.remove(class_index)
        # as this would destructively modify the argument,
        # causing problems during recursive calls

    class_labels_and_counts = Counter([instance[class_index] for instance in instances])

    # If the dataset is empty or the candidate attributes list is empty,
    # return the default value
    if not instances or not candidate_attribute_indexes:
        return default_class

    # If all the instances have the same class label, return that class label
    elif len(class_labels_and_counts) == 1:
        class_label = class_labels_and_counts.most_common(1)[0][0]
        return class_label
    else:
        default_class = majority_value(instances, class_index)

        # Choose the next best attribute index to best classify the instances
        best_index = choose_best_attribute_index(
            instances, candidate_attribute_indexes, class_index)

        # Create a new decision tree node with the best attribute index
        # and an empty dictionary object (for now)
        tree = {(best_index, attribute_names[best_index]): {}}

        # Create a new decision tree sub-node (branch) for each of the values
        # in the best attribute field
        partitions = split_instances(instances, best_index)

        # Remove that attribute from the set of candidates for further splits
        remaining_candidate_attribute_indexes = [i for i in candidate_attribute_indexes
                                                 if i != best_index]
        
        for attribute_value in partitions:
            # Create a subtree for each value of the the best attribute
            subtree = create_decision_tree(
                partitions[attribute_value],
                remaining_candidate_attribute_indexes,
                class_index,
                default_class)

            # Add the new subtree to the empty dictionary object
            # in the new tree/node we just created
            tree[best_index, attribute_names[best_index]][attribute_value] = subtree

    return tree

In [14]:
tree = create_decision_tree(data)
pprint(tree)

{(1, 'Outlook'): {'Overcast': 'Yes',
                  'Rain': {(4, 'Windy'): {'False': 'Yes', 'True': 'No'}},
                  'Sunny': {(3, 'Humidity'): {'High': 'No', 'Normal': 'Yes'}}}}


# Manual Classification

In [15]:
print(attribute_names)
print(new_data)

('Play', 'Outlook', 'Temp', 'Humidity', 'Windy')
('?', 'Rain', 'Mild', 'Normal', 'False')


# Classification

In [16]:
def classify(tree, instance, default_class=None):
    '''Returns a classification label for instance, given a decision tree'''
    
    if not tree:  # if the node is empty, return the default class
        return default_class
    
    if not isinstance(tree, dict):  # if the node is a leaf, return its class label
        return tree
    
    attribute_index = list(tree.keys())[0]  # using list(dict.keys()) for Python 3 compatibility
    attribute_values = list(tree.values())[0]
    instance_attribute_value = instance[attribute_index[0]]
    
    if instance_attribute_value not in attribute_values:  # this value was not in training data
        return default_class
    
    # recursively traverse the subtree (branch) associated with instance_attribute_value
    return classify(attribute_values[instance_attribute_value], instance, default_class)

In [17]:
print("Should he play today? The answer is", classify(tree, new_data))

Should he play today? The answer is Yes


# Analysis

In [18]:
def classification_accuracy(tree, instances, class_index=0, print_predictions=False):
    '''Returns the accuracy of classifying test_instances with tree,
    where the class label is in position class_index'''
    
    ## create a list predicted_labels that contains a prediction for every instance in instances
    predicted_labels = [classify(tree, instance)
                        for instance in instances]
    
    ## create a list actual_labels that contains the real class value for every instance in instances
    actual_labels = [instance[class_index]
                     for instance in instances]

    ## compare the predicted and actual labels for every instance and calculate the accuracy of the predictions
    counts = Counter([x == y
                      for x, y in zip(predicted_labels, actual_labels)])
    
    ## print your result
    print("Accuracy:", counts[True] / len(instances))
    
    if print_predictions:
        for i in range(0, len(instances)):
            predicted_label = predicted_labels[i]
            actual_label = actual_labels[i]
            print('predicted:', predicted_label, 'actual:', actual_label)

In [19]:
def test_analysis(nr_of_train_inst, print_predictions=False):
    nr_of_test_inst = len(data) - nr_of_train_inst
    
    ## split the data into two lists training_instances and test_instances based on the parameter nr_of_train_inst
    training_instances = data[:-nr_of_test_inst]
    test_instances = data[-nr_of_test_inst:]
    ##
    
    ## build a decision tree based on the training instances
    tree = create_decision_tree(training_instances)
    ##
    
    print("Result with", nr_of_train_inst, "training and", nr_of_test_inst, "test instances")
    
    ## call the function classification_accuracy and test the tree with the test_instances
    classification_accuracy(tree, test_instances, print_predictions=print_predictions)
    ##
    
    pprint(tree)
    print()

In [20]:
## call the function test_analysis with different numbers of training instances and determine a good split
## between training and test instances
## if you call the function with print_predictions=True you will see the comparison of predictions and actual labels

test_analysis(6)
test_analysis(10)

Result with 6 training and 8 test instances
Accuracy: 0.75
{(1, 'Outlook'): {'Overcast': 'Yes',
                  'Rain': {(4, 'Windy'): {'False': 'Yes', 'True': 'No'}},
                  'Sunny': 'No'}}

Result with 10 training and 4 test instances
Accuracy: 1.0
{(1, 'Outlook'): {'Overcast': 'Yes',
                  'Rain': {(4, 'Windy'): {'False': 'Yes', 'True': 'No'}},
                  'Sunny': {(3, 'Humidity'): {'High': 'No', 'Normal': 'Yes'}}}}



## Further Reading

[Andy Giese Blog](https://gieseanw.wordpress.com/2012/03/03/decision-tree-learning/)  
[Python for Data Science - 3 Basic Concepts](https://github.com/gumption/Python_for_Data_Science/blob/74bea110cbe47ebecc4d58e32039847a66db5317/3_Python_Basic_Concepts.ipynb)  
[Python for Data Science - 4 Decision Trees](https://github.com/gumption/Python_for_Data_Science/blob/74bea110cbe47ebecc4d58e32039847a66db5317/4_Python_Simple_Decision_Tree.ipynb)